In [14]:
import pandas as pd
import string
import re

In [15]:
# Load dataset
def load_data():
    data = pd.read_csv("digital_korlantas_reviews_5000_cleaned.csv")
    return data

df = load_data()
df.head()

,content,sentiment
0,Mantap,Positif
1,semoga lebih baik lagi,Positif
2,Ok,Positif
3,Sangat membantu dengan adanya pembuatan sim on...,Positif
4,"Alhamdulillah, SIM sudah mendarat dengan selam...",Positif


In [16]:
#import stopword
from nltk.corpus import stopwords 
stopwords_indonesia = stopwords.words('indonesian')
 
#import sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#import tokenize
from nltk.tokenize import word_tokenize 

#stopword tambahan
more_stopword = set(["nya","ya","kali"])

#kamus bahasa tidak baku
slang_word_df = pd.read_csv(r"colloquial-indonesian-lexicon.csv")
slang_word = {}
for i, row in slang_word_df.iterrows():
    slang_word[row["slang"]]=row["formal"]

# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
 
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
 
# all emoticons (happy + sad)
emoticons = emoticons_happy.union(emoticons_sad)

In [17]:
#filtering, stemming, tokenizing
def preprocessing(content):
    print("sebelum ",word_tokenize(content))

    content = content.strip(" ") #menghapus karakter spasi pada awal dan akhir kalimat

    symbols_regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # simbol & piktograf
        u"\U0001F680-\U0001F6FF"  # simbol transport & map
                           "]+", flags = re.UNICODE)

    content = symbols_regrex_pattern.sub(r' ',content) #menghapus emoticon dalam bentuk unicode

    content = content.lower() #ubah menjadi huruf kecil

    content = re.sub(r'^https?:\/\/.*[\r\n]*', ' ', content, flags=re.MULTILINE) # menghapus hyperlink
    
    content = re.sub(r'\n',' ',content) # menghapus baris baru

    content = re.sub('[0-9]+', ' ', content) # menghapus angka
    
    content = re.sub(r'[?|$.!_:")(+,*&%#@]',' ',content) # menghapus beberapa karakter khusus

    content = re.sub(' +', ' ', content) # lebih dari satu spasi yang berdekatan, menjadi satu spasi
 
    tokens = word_tokenize(content) # tokenize kata-kata
 
    content_clean = []    
    for word in tokens: 
        if word in slang_word: # mengganti kata baku dengan yang formal
            word = slang_word[word]

        if (word not in stopwords_indonesia and # menghilangkan stopwords. proses filtering
              word not in emoticons and # menghilangkan emoticons. proses filtering
              word not in more_stopword and
                word not in string.punctuation): # menghilangkan tanda baca. proses filtering
            stem_word = stemmer.stem(word) #  mengubah ke kata dasar. proses stemming
            content_clean.append(stem_word)
 
    print("sesudah ",content_clean)
    print("-----------------------------------------------------------------")
    return content_clean
df['content_preprocessed'] = df['content'].apply(lambda x: preprocessing(x))

sebelum  ['Mantap']
sesudah  ['mantap']
-----------------------------------------------------------------
sebelum  ['semoga', 'lebih', 'baik', 'lagi']
sesudah  ['moga']
-----------------------------------------------------------------
sebelum  ['Ok']
sesudah  ['ok']
-----------------------------------------------------------------
sebelum  ['Sangat', 'membantu', 'dengan', 'adanya', 'pembuatan', 'sim', 'online']
sesudah  ['bantu', 'buat', 'sim', 'online']
-----------------------------------------------------------------
sebelum  ['Alhamdulillah', ',', 'SIM', 'sudah', 'mendarat', 'dengan', 'selamat', 'sampe', 'Rumah', '.', 'Terima', 'Kasih', 'banyak', 'buat', 'Korlantas', 'Polri', '.', 'Mohon', 'maaf', 'jika', 'kemaren', 'agak', 'Pesimis', '.', 'Dengan', 'Aplikasi', 'ini', 'saya', 'sangat', 'terbantu', '.', 'Untuk', 'yg', 'laen', 'Mohon', 'bersabar', ',', 'dari', 'awal', 'daftar', 'mpe', 'SIM', 'mendarat', 'dirumah', 'butuh', 'sekitar', '7', 'Hari', 'kerja', '(', 'krn', 'lonjakan', 'pend

In [18]:
def join_word(text):
    text  = " ".join(text)
    # text  = " ".join([char for char in text if char not in string.punctuation])
    return text
df['content_preprocessed'] = df['content_preprocessed'].apply(lambda x: join_word(x))

In [19]:
sum(df['content_preprocessed']=="") #cek apa bila ada komentar yang kosong setelah proses preprocessing

27

In [20]:
#hilangkan row apabila komentar kosong
nan_value = float("NaN")
df.replace("", nan_value, inplace=True)
df.dropna(subset = ["content_preprocessed"], inplace=True)
sum(df['content_preprocessed']=="")

0

In [21]:
#simpan data yang telah dilakukan preprocessing, untuk selanjutkan akan dilakukan classification
df.to_csv("digital_korlantas_reviews_5000_preprocessed.csv")